Importación de librerías

In [ ]:
import cv2 as cv
import os, sys
from IPython.display import display
from IPython.display import Image as _Imgdis
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
import math
from skimage import io
from math import log10, sqrt
from google.colab import drive

Montamos el Drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Interpolación Bilineal

In [ ]:
def interpolate(subBin,LU,RU,LB,RB,subX,subY):
    subImage = np.zeros(subBin.shape)
    num = subX*subY
    for i in range(subX):
        inverseI = subX-i
        for j in range(subY):
            inverseJ = subY-j
            val = subBin[i,j].astype(int)
            subImage[i,j] = np.floor((inverseI*(inverseJ*LU[val] + j*RU[val])+ i*(inverseJ*LB[val] + j*RB[val]))/float(num))
    return subImage


CLAHE(Ecualización de Histograma Adaptativo Limitado por Contraste)

In [ ]:
def clahe(img,clipLimit,nrBins=128,nrX=0,nrY=0):
    h,w = img.shape
    if clipLimit==1:
        return
    nrBins = max(nrBins,128)
    if nrX==0:
        #Tomar regiones de 32 x 32
        xsz = 32
        ysz = 32
        nrX = math.ceil(h/xsz)#240
        #Exceso de píxeles para obtener un valor entero de nrX y nrY
        excX= int(xsz*(nrX-h/xsz))
        nrY = math.ceil(w/ysz)#320
        excY= int(ysz*(nrY-w/ysz))
        if excX!=0:
            img = np.append(img,np.zeros((excX,img.shape[1])).astype(int),axis=0)
        if excY!=0:
            img = np.append(img,np.zeros((img.shape[0],excY)).astype(int),axis=1)
    else:
        xsz = round(h/nrX)
        ysz = round(w/nrY)
    
    nrPixels = xsz*ysz
    xsz2 = round(xsz/2)
    ysz2 = round(ysz/2)
    claheimg = np.zeros(img.shape)
    
    if clipLimit > 0:
        clipLimit = max(1,clipLimit*xsz*ysz/nrBins)
    else:
        clipLimit = 50
    
    
    minVal = 0 #np.min(img)
    maxVal = 255 #np.max(img)
    binSz = np.floor(1+(maxVal-minVal)/float(nrBins))
    LUT = np.floor((np.arange(minVal,maxVal+1)-minVal)/float(binSz))
    
    bins = LUT[img]
    #Creacion del histograma
    hist = np.zeros((nrX,nrY,nrBins))
    #print(nrX,nrY,hist.shape)
    for i in range(nrX):
        for j in range(nrY):
            bin_ = bins[i*xsz:(i+1)*xsz,j*ysz:(j+1)*ysz].astype(int)
            for i1 in range(xsz):
                for j1 in range(ysz):
                    hist[i,j,bin_[i1,j1]]+=1
    
    #clipHistogram
    if clipLimit>0:
        for i in range(nrX):
            for j in range(nrY):
                nrExcess = 0
                for nr in range(nrBins):
                    excess = hist[i,j,nr] - clipLimit
                    if excess>0:
                        nrExcess += excess
                
                binIncr = nrExcess/nrBins
                upper = clipLimit - binIncr
                for nr in range(nrBins):
                    if hist[i,j,nr] > clipLimit:
                        hist[i,j,nr] = clipLimit
                    else:
                        if hist[i,j,nr]>upper:
                            nrExcess += upper - hist[i,j,nr]
                            hist[i,j,nr] = clipLimit
                        else:
                            nrExcess -= binIncr
                            hist[i,j,nr] += binIncr
                
                if nrExcess > 0:
                    stepSz = max(1,np.floor(1+nrExcess/nrBins))
                    for nr in range(nrBins):
                        nrExcess -= stepSz
                        hist[i,j,nr] += stepSz
                        if nrExcess < 1:
                            break
    
    #mapeo de histograma
    map_ = np.zeros((nrX,nrY,nrBins))
    scale = (maxVal - minVal)/float(nrPixels)
    for i in range(nrX):
        for j in range(nrY):
            sum_ = 0
            for nr in range(nrBins):
                sum_ += hist[i,j,nr]
                map_[i,j,nr] = np.floor(min(minVal+sum_*scale,maxVal))
    
    
    #interpolacion
    xI = 0
    for i in range(nrX+1):
        if i==0:
            subX = int(xsz/2)
            xU = 0
            xB = 0
        elif i==nrX:
            subX = int(xsz/2)
            xU = nrX-1
            xB = nrX-1
        else:
            subX = xsz
            xU = i-1
            xB = i
        
        yI = 0
        for j in range(nrY+1):
            if j==0:
                subY = int(ysz/2)
                yL = 0
                yR = 0
            elif j==nrY:
                subY = int(ysz/2)
                yL = nrY-1
                yR = nrY-1
            else:
                subY = ysz
                yL = j-1
                yR = j
            UL = map_[xU,yL,:]
            UR = map_[xU,yR,:]
            BL = map_[xB,yL,:]
            BR = map_[xB,yR,:]
            subBin = bins[xI:xI+subX,yI:yI+subY]
            subImage = interpolate(subBin,UL,UR,BL,BR,subX,subY)
            claheimg[xI:xI+subX,yI:yI+subY] = subImage
            yI += subY
        xI += subX
    
    if excX==0 and excY!=0:
        return claheimg[:,:-excY]
    elif excX!=0 and excY==0:
        return claheimg[:-excX,:]
    elif excX!=0 and excY!=0:
        return claheimg[:-excX,:-excY]
    else:
        return claheimg

In [ ]:
def histogram(inp,h):
    s=0
    acum=0
    tamima=inp.size
    tam=len(h)
    L=256
    V=[]
    for i in range(tam):
        acum=acum+(h[i]/tamima)
        s=math.floor((L-1)*acum)
        V.append(s)
    return V
def equalization(inp,h):
    nh=histogram(inp,h)
    f,c=inp.shape
    for i in range(f):
        for j in range(c):
            col=inp[i][j]
            inp[i][j]=nh[col]
    return inp

Ecualización de Histograma tradicional

In [ ]:
def PSNR(original, compressed): 
    mse = np.mean((original - compressed) ** 2) 
    if(mse == 0):    
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse)) 
    return psnr
def MSE(original, compressed): 
    mse = np.mean((original - compressed) ** 2)
    return mse 

Realización de pruebas y resultados, cabe tener en cuenta que para el filtro mediano se uso una función de OpenCV.

In [ ]:
#Ecualizacion de histograma y filtro mediano
#Abrir imagenes
folder = "/content/drive/My Drive/rxneumo"
onlyfiles = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
#Abrir imagenes
psnrehm=0
msehm=0
psnrclahe = 0
mseclahe = 0
tam = 175
print("tamaño de muestra: "+str(tam))
for i in range(tam):
  s = "/content/drive/My Drive/rxneumo/"+onlyfiles[i]
  print(str(i)+" "+onlyfiles[i])
  a = cv.imread(s,0)
  original = cv.imread(s,0)
  hi = cv.calcHist([a], [0], None, [256], [0, 256])
  median_eh = cv.medianBlur(a,3)
  imagen_eh=equalization(median_eh,hi)
  #plt.hist(a.ravel(),256,[0,256])
  #plt.xlabel("Color de los píxeles")
  #plt.ylabel("Número de píxeles")
  #plt.show()
  #CLAHE
  #print("Resultados PSNR y MSE para imagen con ecualizacion de histograma y filtro mediano")
  psnrehm = psnrehm + PSNR(original,imagen_eh)
  msehm = msehm + MSE(original,imagen_eh)
  #print("PSNR: "+str(psnrehm))
  #print("MSE: "+str(msehm))
  image = io.imread(s)
  clahe_img = clahe(image,8,0,0)
  #print("Resultados PSNR y MSE para imagen con CLAHE")
  psnrclahe = psnrclahe+PSNR(original,clahe_img)
  mseclahe = mseclahe+MSE(original,clahe_img)
  #print("PSNR: "+str(psnrclahe))
  #print("MSE: "+str(mseclahe))
  #plt.subplot(1,3,1)
  #original = cv.resize(original,(800,800))
  #plt.imshow(original,cmap = 'gray')    
  #plt.title('Original')
  #plt.subplot(1,3,2)
  #median_eh = cv.resize(median_eh,(800,800))
  #plt.imshow(median_eh,cmap = 'gray')  
  #plt.title('Ecualización de Histograma + filtro mediano')
  #plt.subplot(1,3,3)
  #clahe_img = cv.resize(clahe_img,(800,800))
  #plt.imshow(clahe_img,cmap = 'gray')     
  #plt.title('CLAHE')
  #plt.show()

print("Resultados(promedio) PSNR y MSE para imagen con ecualizacion de histograma y filtro mediano (radiografias con neumonia)")
print("PSNR: "+str(psnrehm/tam))
print("MSE: "+str(msehm/tam))
print("Resultados(promedio) PSNR y MSE para imagen con CLAHE (radiografias con neumonia)")
print("PSNR: "+str(psnrclahe/tam))
print("MSE: "+str(mseclahe/tam))

tamaño de muestra: 175
0 person1343_virus_2317.jpeg
1 person1349_bacteria_3438.jpeg
2 person1348_virus_2326.jpeg
3 person1344_bacteria_3421.jpeg
4 person1351_virus_2330.jpeg
5 person1349_bacteria_3437.jpeg
6 person1344_virus_2320.jpeg
7 person1349_bacteria_3434.jpeg
8 person1350_virus_2329.jpeg
9 person1344_virus_2319.jpeg
10 person1346_virus_2322.jpeg
11 person1345_bacteria_3424.jpeg
12 person1349_bacteria_3439.jpeg
13 person1346_bacteria_3430.jpeg
14 person1345_bacteria_3427.jpeg
15 person1347_virus_2323.jpeg
16 person1348_virus_2324.jpeg
17 person1345_bacteria_3422.jpeg
18 person1349_bacteria_3436.jpeg
19 person1351_bacteria_3441.jpeg
20 person1352_bacteria_3442.jpeg
21 person1345_virus_2321.jpeg
22 person1345_bacteria_3428.jpeg
23 person1345_bacteria_3426.jpeg
24 person1345_bacteria_3425.jpeg
25 person1352_bacteria_3443.jpeg
26 person1352_bacteria_3444.jpeg
27 person1353_bacteria_3446.jpeg
28 person1352_bacteria_3445.jpeg
29 person1374_virus_2365.jpeg
30 person1358_virus_2339.jpeg
